In [ ]:
 pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math
import tempfile
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau


# Modifying Data

In [ ]:
!ls 
# !unzip sign.zip

american_sign_language.PNG  sample_data		 sign_mnist_train
amer_sign2.png		    sign_mnist_test	 sign_mnist_train.csv
amer_sign3.png		    sign_mnist_test.csv  sign.zip


In [ ]:
train_df = pd.read_csv("sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")
# print(test_df)

In [ ]:
test = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")
# print(test)
y = test['label']
# print(y.max())

In [ ]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,156,158,160,163,165,159,166,168,170,170,171,171,171,172,171,171,170,170,169,111,121,129,135,141,144,148,151,154,157,160,...,205,206,206,207,207,206,206,204,205,204,203,202,142,151,160,172,196,188,188,190,135,96,86,77,77,79,176,205,207,207,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,157,158,156,154,154,153,152,151,149,149,148,147,146,144,142,143,138,92,108,158,159,159,159,160,160,160,160,160,160,160,...,100,78,120,157,168,107,99,121,133,97,95,120,135,116,95,79,69,86,139,173,200,185,175,198,124,118,94,140,133,84,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,186,185,185,185,184,184,184,181,181,179,179,179,178,178,109,52,66,77,83,188,189,189,188,188,189,188,188,188,188,187,...,203,204,203,201,200,200,199,198,196,195,194,193,198,166,132,114,89,74,79,77,74,78,132,188,210,209,206,205,204,203,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,211,209,207,208,207,206,203,202,201,200,198,197,195,192,197,171,51,52,54,212,213,215,215,212,212,213,212,212,211,211,...,247,242,233,231,230,229,227,225,223,221,220,216,58,51,49,50,57,60,17,15,18,17,19,1,159,255,237,239,237,236,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,186,188,189,189,190,191,189,190,190,187,190,192,193,191,191,192,192,194,194,166,169,172,174,177,180,182,185,186,187,190,...,90,77,88,117,123,127,129,134,145,152,156,179,105,106,105,104,104,104,175,199,178,152,136,130,136,150,118,92,85,76,92,105,105,108,133,163,157,163,164,179


In [ ]:
train_labels = train_df['label']
test_labels = test_df['label']
del train_df['label']
del test_df['label']

In [ ]:
train_images = train_df.values
test_images = test_df.values
# Normalize the data

# print(train_images.shape)
# print(test_images.shape)

train_images = train_images / 255
test_images = test_images / 255
train_images = train_images.reshape(-1,28,28,1)
test_images = test_images.reshape(-1,28,28,1)
# print(train_images.shape)
# print(test_images.shape)

# print(train_df)
# print(train_images)

# Creating CNN model

In [ ]:

#setup CNN model
def setup_model():
  model = keras.Sequential([
      # tf.keras.layers.Reshape(input_shape=(28*28,), target_shape(28,28,1)),

      keras.layers.Conv2D(kernel_size = 3, filters = 100, activation = 'relu', padding = 'same', input_shape=(28,28,1)), 
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),

      keras.layers.Conv2D(kernel_size = 3, filters = 70, activation = 'relu', padding = 'same', strides = 1),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),
      tf.keras.layers.MaxPool2D((2,2) , strides = 1 , padding = 'same'),
      keras.layers.Dropout(0.25),

      keras.layers.Conv2D(kernel_size = 3, filters = 32, activation = 'relu', padding = 'same', strides = 1),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),



      keras.layers.Flatten(),

      keras.layers.Dense(500, use_bias = False),
      tf.keras.layers.BatchNormalization(center = True, scale = False),
      tf.keras.layers.Activation('relu'),
      keras.layers.Dropout(0.20),

      keras.layers.Dense(100, activation = 'relu'),
      keras.layers.Dropout(0.15),

      keras.layers.Dense(25, activation = 'softmax')
      ])
  return model

In [ ]:
#lr decay function
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

#lr schedule callback
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay,)

In [ ]:
def train_model(model):
  model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
  model.summary()
  model.fit(train_images, train_labels, epochs=20, callbacks = [lr_decay_callback])
  return model

In [ ]:
def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

In [ ]:
def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

In [ ]:
def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

In [ ]:
def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

In [ ]:
#define the model
model = setup_model()
# Train the model
model = train_model(model)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 100)       1000      
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 100)       300       
_________________________________________________________________
activation_12 (Activation)   (None, 28, 28, 100)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 70)        63070     
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 70)        210       
_________________________________________________________________
activation_13 (Activation)   (None, 28, 28, 70)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 70)       

In [ ]:
# Evaluate the base model
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9380925893783569
Saving model to:  /tmp/tmpxq7ta9l_.h5
